# The Chocolate dataset: Different multi-class 
https://nilearn.github.io/auto_examples/02_decoding/plot_haxby_multiclass.html#sphx-glr-auto-examples-02-decoding-plot-haxby-multiclass-py

In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
import nilearn
import glob

In [ ]:
basepath=os.path.join('/projects','niblab','data','eric_data','W1','imagine')
#basepath ='/projects/niblab/nilearn_projects'
outpath = "/projects/niblab/nilearn_projects"
#make a list of the files to concat
all_func = glob.glob(os.path.join(basepath,'level1_grace_edit','cs*++.feat','filtered_func_data.nii.gz'))
len(all_func)

fmri_subjs=os.path.join(outpath, 'concatenated_imagine_all.nii')
average_ana=os.path.join(outpath,'CS_avg_mprage_image.nii.gz')
imag_mask=os.path.join(outpath,'power_roimask_4bi.nii.gz')
#imag_mask="/projects/niblab/data/eric_data/W1/imagine/level1_grace_edit/cs001++.feat/mask.nii.gz"
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
labels = pd.read_csv(stim, sep=",")
y =  labels['label']
session = labels['sub']

# removing 'trash' and 'rest'
non_trash = (y != 'trash')
y = y[non_trash]
non_rest = (y != 'rest')
y = y[non_rest]

# Get the labels of the numerical conditions represented by the vector y
unique_conditions, order = np.unique(y, return_index=True)
# Sort the conditions by the order of appearance
unique_conditions = unique_conditions[np.argsort(order)]
#output > 
#array(['unapp', 'H2O', 'app'], dtype=object)


from nilearn.input_data import NiftiMasker
# For decoding, standardizing is often very important
nifti_masker = NiftiMasker(mask_img=imag_mask, standardize=True)
X = nifti_masker.fit_transform(fmri_subjs)

# Remove the "rest" conditionv --make sure to do this in the same order you did for y 
# otherwise you will run into dimension error 
X = X[non_trash]
session = session[non_trash]
X = X[non_rest]
session = session[non_rest]



## Build the decoders, using scikit-learn  
We are using a Support Vector Classification below,  
we will try a linear kernel, ...,  
and a simple feature selection step.  

In [ ]:

from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.pipeline import Pipeline

svc_ovo = OneVsOneClassifier(Pipeline([
    ('anova', SelectKBest(f_classif, k=500)),
    ('svc', SVC(kernel='linear'))
]))

svc_ova = OneVsRestClassifier(Pipeline([
    ('anova', SelectKBest(f_classif, k=500)),
    ('svc', SVC(kernel='linear'))
]))



## Compute Cross-validation scores

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores_ovo = cross_val_score(svc_ovo, X, y, cv=5, verbose=1)

cv_scores_ova = cross_val_score(svc_ova, X, y, cv=5, verbose=1)

print('OvO:', cv_scores_ovo.mean())
print('OvA:', cv_scores_ova.mean())